In [112]:
import pandas as pd
url = 'https://github.com/mattharrison/datasets/raw/master/data/siena2018-pres.csv'

In [113]:
import pandas as pd

In [114]:
df = pd.read_csv(url, index_col=0, dtype_backend='pyarrow')

def tweak_siena_pres(df):
    def int64_to_uint8(df_):
        cols = df_.select_dtypes('int64')
        return (df_
                .astype({col:'uint8[pyarrow]' for col in cols}))
    return (df
            .rename(columns={'Seq.': 'Seq'}) # 1
            .rename(columns={k:v.replace(' ', '_') for k,v in
                             {'Bg': 'Background',
                              'PL': 'Party leadership', 'CAb': 'Communication ability',
                              'RC': 'Relations with Congress', 'CAp': 'Court appointments',
                              'HE': 'Handling of economy', 'L': 'Luck',
                              'AC': 'Ability to compromise', 'WR': 'Willing to take risks',
                              'EAp': 'Executive appointments', 'OA': 'Overall ability',
                              'Im': 'Imagination', 'DA': 'Domestic accomplishments',
                              'Int': 'Integrity', 'EAb': 'Executive ability',
                              'FPA': 'Foreign policy accomplishments',
                              'LA': 'Leadership ability',
                              'IQ': 'Intelligence', 'AM': 'Avoid crucial mistakes',
                              'EV': "Experts' view", 'O': 'Overall'}.items()})
                              .astype({'Party':'category'})  # 2
                              .pipe(int64_to_uint8)  # 3
                              .assign(Average_rank=lambda df_:(df_.select_dtypes('uint8')
                                                               .sum(axis=1).rank(method='dense').astype('uint8[pyarrow]')),
                                                               Quartile=lambda df_:pd.qcut(df_.Average_rank, 4,
                                                                                           labels='1st 2nd 3rd 4th'.split())
                                    )
    )


pres = tweak_siena_pres(df)

In [115]:
def tweak_siena_pres(df):
    def int64_to_uint8(df_):
        cols = df_.select_dtypes('int64')
        return (df_
                .astype({col:'uint8[pyarrow]' for col in cols}))
    return (df
            .rename(columns={'Seq.': 'Seq'}) # 1
            .rename(columns={k:v.replace(' ', '_') for k,v in
                             {'Bg': 'Background',
                              'PL': 'Party leadership', 'CAb': 'Communication ability',
                              'RC': 'Relations with Congress', 'CAp': 'Court appointments',
                              'HE': 'Handling of economy', 'L': 'Luck',
                              'AC': 'Ability to compromise', 'WR': 'Willing to take risks',
                              'EAp': 'Executive appointments', 'OA': 'Overall ability',
                              'Im': 'Imagination', 'DA': 'Domestic accomplishments',
                              'Int': 'Integrity', 'EAb': 'Executive ability',
                              'FPA': 'Foreign policy accomplishments',
                              'LA': 'Leadership ability',
                              'IQ': 'Intelligence', 'AM': 'Avoid crucial mistakes',
                              'EV': "Experts' view", 'O': 'Overall'}.items()})
                              .astype({'Party':'category'})  # 2
                              .pipe(int64_to_uint8)  # 3
                              .assign(Average_rank=lambda df_:(df_.select_dtypes('uint8')
                                                               .sum(axis=1).rank(method='dense').astype('uint8[pyarrow]')),
                                                               Quartile=lambda df_:pd.qcut(df_.Average_rank, 4,
                                                                                           labels='1st 2nd 3rd 4th'.split())
                                    )
    )


In [116]:
# interaction over columns (col_name, series) tuple
for col_name, col in pres.items():
    print(col_name, type(col))
    break

Seq <class 'pandas.core.series.Series'>


In [117]:
scores = (pres
          .loc[:,'Background':'Average_rank']
)
scores.sum(axis='columns') / len(scores.columns)

1      3.681818
2     14.454545
3      6.545455
4      9.636364
5     10.454545
6     17.181818
7     19.590909
8     25.681818
9     36.909091
10    34.409091
11    13.318182
12         29.5
13    37.454545
14    39.409091
15         42.0
16     4.045455
17    42.272727
18    24.227273
19    30.136364
20    27.272727
21    31.454545
22    22.181818
23    32.818182
24    19.727273
25     5.227273
26    21.318182
27    13.590909
28    38.772727
29    29.909091
30    31.954545
31     3.909091
32    11.818182
33     9.227273
34    12.727273
35    15.272727
36    26.909091
37         26.0
38    26.818182
39    14.545455
40    20.818182
41    14.636364
42    30.363636
43    15.818182
44    39.772727
dtype: double[pyarrow]

In [118]:
pres.select_dtypes('number').agg(
    ['count','size','sum', lambda col: col.loc[1]]
)

,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,...,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank
count,44,44,44,44,44,44,44,44,44,44,...,44,44,44,44,44,44,44,44,44,44
size,44,44,44,44,44,44,44,44,44,44,...,44,44,44,44,44,44,44,44,44,44
sum,968,957,990,990,990,953,968,978,990,990,...,979,990,990,990,990,990,990,990,990,990
<lambda>,7,7,1,10,1,6,2,2,1,11,...,1,1,1,1,2,2,1,2,1,1


In [119]:
# Using a dictionary to create a pivot chart equivalent
pres.agg(
    {'Luck':['count','size'],
     'Overall': ['count', 'max']}
)

,Luck,Overall
count,44.0,44.0
size,44.0,NaN
max,NaN,44.0


In [120]:
# First, specify the row you want, and within the brackets, specify the aggregations you want
pres.agg(Intelligence_count=('Intelligence','count'),
         Intelligence_size=('Intelligence','size'))

,Intelligence
Intelligence_count,44
Intelligence_size,44


In [121]:
pres.describe()

,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,...,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank
count,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,...,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0
mean,22.0,21.75,22.5,22.5,22.5,21.659091,22.0,22.227273,22.5,22.5,...,22.25,22.5,22.5,22.5,22.5,22.5,22.5,22.5,22.5,22.5
std,12.409674,12.519984,12.845233,12.845233,12.845233,11.892822,12.409674,12.500909,12.845233,12.845233,...,12.519984,12.845233,12.845233,12.845233,12.845233,12.845233,12.845233,12.845233,12.845233,12.845233
min,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
25%,11.75,11.0,11.75,11.75,11.75,11.75,11.75,11.75,11.75,11.75,...,11.75,11.75,11.75,11.75,11.75,11.75,11.75,11.75,11.75,11.75
50%,22.0,21.5,22.5,22.5,22.5,22.5,22.0,22.5,22.5,22.5,...,22.5,22.5,22.5,22.5,22.5,22.5,22.5,22.5,22.5,22.5
75%,32.25,32.25,33.25,33.25,33.25,31.25,32.25,32.25,33.25,33.25,...,33.0,33.25,33.25,33.25,33.25,33.25,33.25,33.25,33.25,33.25
max,43.0,43.0,44.0,44.0,44.0,41.0,43.0,43.0,44.0,44.0,...,43.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0,44.0


In [122]:
# (pres
#  .select_dtypes('columns')
#  .pipe(lambda df_: df_.max(axis=1) - df_.min(axis=0))
#  .rename('Range')
#  )

In [123]:
(pres
 .select_dtypes('number')
 .apply(lambda row: row.max()-row.min(), axis='columns')
 .rename('range')
)

1     17
2     28
3     19
4     16
5     13
6     28
7     34
8     18
9     22
10    19
11    16
12    15
13     8
14     3
15     8
16    27
17    10
18    21
19    13
20    21
21    24
22    12
23     8
24    21
25    13
26    19
27    28
28    10
29    26
30    31
31    15
32    27
33    18
34    28
35    38
36    31
37    23
38    35
39    28
40    19
41    36
42    24
43    22
44    34
Name: range, dtype: int64

In [124]:
pres.select_dtypes('number').apply('sum')

Background                        968
Imagination                       957
Integrity                         990
Intelligence                      990
Luck                              990
Willing_to_take_risks             953
Ability_to_compromise             968
Executive_ability                 978
Leadership_ability                990
Communication_ability             990
Overall_ability                   990
Party_leadership                  990
Relations_with_Congress           979
Court_appointments                990
Handling_of_economy               990
Executive_appointments            990
Domestic_accomplishments          990
Foreign_policy_accomplishments    990
Avoid_crucial_mistakes            990
Experts'_view                     990
Overall                           990
Average_rank                      990
dtype: uint64[pyarrow]

In [125]:
pres.select_dtypes('number').sum()

Background                        968
Imagination                       957
Integrity                         990
Intelligence                      990
Luck                              990
Willing_to_take_risks             953
Ability_to_compromise             968
Executive_ability                 978
Leadership_ability                990
Communication_ability             990
Overall_ability                   990
Party_leadership                  990
Relations_with_Congress           979
Court_appointments                990
Handling_of_economy               990
Executive_appointments            990
Domestic_accomplishments          990
Foreign_policy_accomplishments    990
Avoid_crucial_mistakes            990
Experts'_view                     990
Overall                           990
Average_rank                      990
dtype: uint64[pyarrow]

In [126]:
import io

In [127]:
billing_data = \
'''cancel_date,period_start,start_date,end_date,rev,sum_payments
12/1/2019,1/1/2020,12/15/2019,5/15/2020,999,50
,1/1/2020,12/15/2019,5/15/2020,999,50
,1/1/2020,12/15/2019,5/15/2020,999,1950
1/20/2020,1/1/2020,12/15/2019,5/15/2020,499,0
,1/1/2020,12/24/2019,5/24/2020,699,100
,1/1/2020,11/29/2019,4/29/2020,799,250
,1/1/2020,1/15/2020,4/29/2020,799,250'''

bill_df = pd.read_csv(io.StringIO(billing_data),
                      dtype_backend='pyarrow',
                      parse_dates=['cancel_date','period_start','start_date','end_date'])

In [128]:
bill_df

,cancel_date,period_start,start_date,end_date,rev,sum_payments
0,12/1/2019,2020-01-01,2019-12-15,2020-05-15,999,50
1,<NA>,2020-01-01,2019-12-15,2020-05-15,999,50
2,<NA>,2020-01-01,2019-12-15,2020-05-15,999,1950
3,1/20/2020,2020-01-01,2019-12-15,2020-05-15,499,0
4,<NA>,2020-01-01,2019-12-24,2020-05-24,699,100
5,<NA>,2020-01-01,2019-11-29,2020-04-29,799,250
6,<NA>,2020-01-01,2020-01-15,2020-04-29,799,250


In [129]:
def tweak_bill202(df_):
    return (df_
            .assign(cancel_date=pd.to_datetime(
                df_.cancel_date.replace('<NA>',''), format='%m/%d/%Y')
                ))


bill_df = tweak_bill202(bill_df)
bill_df

,cancel_date,period_start,start_date,end_date,rev,sum_payments
0,2019-12-01,2020-01-01,2019-12-15,2020-05-15,999,50
1,NaT,2020-01-01,2019-12-15,2020-05-15,999,50
2,NaT,2020-01-01,2019-12-15,2020-05-15,999,1950
3,2020-01-20,2020-01-01,2019-12-15,2020-05-15,499,0
4,NaT,2020-01-01,2019-12-24,2020-05-24,699,100
5,NaT,2020-01-01,2019-11-29,2020-04-29,799,250
6,NaT,2020-01-01,2020-01-15,2020-04-29,799,250


In [130]:
bill_df = tweak_bill202(bill_df)

In [131]:
def tweak_billing(df_):
    return (df_
            .assign(cancel_date=pd.to_datetime(
                df_.cancel_date.replace('<NA>', format='m%/%d/%Y')
            )))

bill_df = tweak_bill202(bill_df)
bill_df

,cancel_date,period_start,start_date,end_date,rev,sum_payments
0,2019-12-01,2020-01-01,2019-12-15,2020-05-15,999,50
1,NaT,2020-01-01,2019-12-15,2020-05-15,999,50
2,NaT,2020-01-01,2019-12-15,2020-05-15,999,1950
3,2020-01-20,2020-01-01,2019-12-15,2020-05-15,499,0
4,NaT,2020-01-01,2019-12-24,2020-05-24,699,100
5,NaT,2020-01-01,2019-11-29,2020-04-29,799,250
6,NaT,2020-01-01,2020-01-15,2020-04-29,799,250


In [132]:
import numpy as np

In [133]:
def calc_unbilled_rec(vals):
    cancel_date, period_start, start_date, end_date, rev, sum_payments = vals
    if cancel_date < period_start:
        return np.nan
    if start_date < period_start and end_date > period_start:
        if rev > sum_payments:
            return rev - sum_payments
        else:
            return

In [134]:
def calc_unbilled_rec(vals):
    cancel_date, period_start, start_date, end_date, rev, sum_payments = vals # first defines everything
    if cancel_date < period_start: # if cancel_date is lower than period_start
        return np.nan # return nan. Why though
    if start_date < period_start and end_date > period_start:
        if rev > sum_payments:
            return rev - sum_payments
        else:
            return

In [135]:
bill_df.apply(calc_unbilled_rec, axis='columns')

0      NaN
1    949.0
2      NaN
3    499.0
4    599.0
5    549.0
6      NaN
dtype: float64

In [136]:
(pd.Series(np.nan, dtype='float[pyarrow]', index=bill_df.index)
 .case_when([(bill_df.cancel_date < bill_df.period_start, np.nan),
             (((bill_df.start_date < bill_df.period_start) & # 2
               (bill_df.end_date > bill_df.period_start) &
               (bill_df.rev > bill_df.sum_payments)), 
               bill_df.rev - bill_df.sum_payments), # condition result 2
               (((bill_df.start_date < bill_df.period_start) & # 3
                 (bill_df.end_date > bill_df.period_start) & 
                 (bill_df.rev <= bill_df.sum_payments)), 
                 0)]) # condition result 3
)

0     <NA>
1    949.0
2      0.0
3    499.0
4    599.0
5    549.0
6     <NA>
dtype: double[pyarrow]

In [137]:
bill_100k = bill_df.sample(100_000, replace=True)

In [138]:
bill_100k

,cancel_date,period_start,start_date,end_date,rev,sum_payments
0,2019-12-01,2020-01-01,2019-12-15,2020-05-15,999,50
2,NaT,2020-01-01,2019-12-15,2020-05-15,999,1950
4,NaT,2020-01-01,2019-12-24,2020-05-24,699,100
6,NaT,2020-01-01,2020-01-15,2020-04-29,799,250
2,NaT,2020-01-01,2019-12-15,2020-05-15,999,1950
...,...,...,...,...,...,...
2,NaT,2020-01-01,2019-12-15,2020-05-15,999,1950
2,NaT,2020-01-01,2019-12-15,2020-05-15,999,1950
3,2020-01-20,2020-01-01,2019-12-15,2020-05-15,499,0
3,2020-01-20,2020-01-01,2019-12-15,2020-05-15,499,0


In [139]:
%%timeit
bill_100k.apply(calc_unbilled_rec, axis='columns')

203 ms ± 5.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [140]:
def calc_unbilled_case(bill_df):
    return (pd.Series(np.nan, dtype='float[pyarrow]', index=bill_df.index)
            .case_when([(bill_df.cancel_date < bill_df.period_start, # when condition 1
                        np.nan), # result 1
                        (((bill_df.start_date < bill_df.period_start) & # when condition 2.1
                          (bill_df.end_date > bill_df.period_start) & # when condition 2.2
                          (bill_df.rev > bill_df.sum_payments)), # when condition 2.3
                          bill_df.rev - bill_df.sum_payments),# result 2

                          (((bill_df.start_date > bill_df.period_start) & # when condition 3.1
                            (bill_df.end_date > bill_df.period_start) & # when condition 3.2
                            (bill_df.rev <= bill_df.sum_payments)), # when condition 3.3
                            0) # result 3
                            ])
                            )

In [141]:
%%timeit
calc_unbilled_case(bill_100k)

2.58 ms ± 63.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [142]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [143]:
%%cython

def calc_unbilled_rec_cy1(vals):
    cancel_date, period_start, start_date, end_date, rev, sum_payments = vals
    if cancel_date < period_start:
        return float('nan')
    if start_date < period_start and end_date > period_start:
        if rev > sum_payments:
            return rev - sum_payments
        else:
            return 0

In [144]:
%%timeit
bill_100k.apply(calc_unbilled_rec_cy1, axis='columns')

198 ms ± 4.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [145]:
import numpy as np
from numba import jit

@jit
def calc_unbilled_numba(cancel_date, period_start, start_date, end_date, rev, sum_payments):
    results = np.full(rev.shape[0], np.nan, dtype=np.float64)
    for i in range(rev.shape[0]):
        cd = cancel_date[i]
        ps = period_start[i]
        sd = start_date[i]
        ed = end_date[i]
        if cd > 0 and cd < ps:
            results[i] = np.nan
        elif sd < ps < ed:
            if rev[i] > sum_payments[i]:
                results[i] = rev[i] - sum_payments[i]
            else:
                results[i] = 0
        return results

In [146]:
%%timeit
calc_unbilled_numba(*(ser.astype(int).to_numpy() for name, ser in bill_100k.items()))

184 μs ± 4.13 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [147]:
# Exercises

In [148]:
bill_100k

,cancel_date,period_start,start_date,end_date,rev,sum_payments
0,2019-12-01,2020-01-01,2019-12-15,2020-05-15,999,50
2,NaT,2020-01-01,2019-12-15,2020-05-15,999,1950
4,NaT,2020-01-01,2019-12-24,2020-05-24,699,100
6,NaT,2020-01-01,2020-01-15,2020-04-29,799,250
2,NaT,2020-01-01,2019-12-15,2020-05-15,999,1950
...,...,...,...,...,...,...
2,NaT,2020-01-01,2019-12-15,2020-05-15,999,1950
2,NaT,2020-01-01,2019-12-15,2020-05-15,999,1950
3,2020-01-20,2020-01-01,2019-12-15,2020-05-15,499,0
3,2020-01-20,2020-01-01,2019-12-15,2020-05-15,499,0


In [149]:
df = pd.DataFrame({"a":[1,2], "b":[3,4]})
df.max(axis=0)

a    2
b    4
dtype: int64

In [150]:
df

,a,b
0,1,3
1,2,4


In [151]:
# Loop over each row and calculate the maximum and minimum values

pres_simple = pres.copy()

pres_simple['Max'] = (pres_simple
               .max(axis=1, skipna=True, numeric_only=True))
pres_simple['Min'] = (pres_simple
               .min(axis=1, skipna=True, numeric_only=True))



pres_simple

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile,Max,Min
1,1,George Washington,Independent,7,7,1,10,1,6,2,...,1,2,2,1,2,1,1,1st,18,1
2,2,John Adams,Federalist,3,13,4,4,24,14,31,...,15,19,13,16,10,14,13,2nd,31,3
3,3,Thomas Jefferson,Democratic-Republican,2,2,14,1,8,5,14,...,4,6,9,7,5,5,5,1st,20,1
4,4,James Madison,Democratic-Republican,4,6,7,3,16,15,6,...,7,11,19,11,8,7,7,1st,19,3
5,5,James Monroe,Democratic-Republican,9,14,11,18,6,16,7,...,9,10,5,6,9,8,8,1st,18,5
6,6,John Quincy Adams,Democratic-Republican,1,9,6,5,29,19,24,...,18,21,15,14,18,18,18,2nd,29,1
7,7,Andrew Jackson,Democratic,37,15,29,28,4,4,38,...,25,17,23,20,19,19,19,2nd,38,4
8,8,Martin Van Buren,Democratic,23,22,27,25,34,28,20,...,26,29,27,24,28,25,25,3rd,34,16
9,9,William Henry Harrison,Whig,22,38,28,37,44,32,41,...,40,42,44,37,39,39,38,4th,44,22
10,10,John Tyler,Independent,34,33,35,34,22,26,37,...,36,36,26,32,36,37,37,4th,41,22


In [152]:
# Calculate each row and column’s maximum and minimum value using the .agg method.
pres_agg = pres.copy()

row_results = (pres_agg
                     .select_dtypes(include='number')
                     .agg(['min','max'], axis=1))


pres_agg[['Row_min', 'Row_max']] = row_results
pres_agg


,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile,Row_min,Row_max
1,1,George Washington,Independent,7,7,1,10,1,6,2,...,1,2,2,1,2,1,1,1st,1,18
2,2,John Adams,Federalist,3,13,4,4,24,14,31,...,15,19,13,16,10,14,13,2nd,3,31
3,3,Thomas Jefferson,Democratic-Republican,2,2,14,1,8,5,14,...,4,6,9,7,5,5,5,1st,1,20
4,4,James Madison,Democratic-Republican,4,6,7,3,16,15,6,...,7,11,19,11,8,7,7,1st,3,19
5,5,James Monroe,Democratic-Republican,9,14,11,18,6,16,7,...,9,10,5,6,9,8,8,1st,5,18
6,6,John Quincy Adams,Democratic-Republican,1,9,6,5,29,19,24,...,18,21,15,14,18,18,18,2nd,1,29
7,7,Andrew Jackson,Democratic,37,15,29,28,4,4,38,...,25,17,23,20,19,19,19,2nd,4,38
8,8,Martin Van Buren,Democratic,23,22,27,25,34,28,20,...,26,29,27,24,28,25,25,3rd,16,34
9,9,William Henry Harrison,Whig,22,38,28,37,44,32,41,...,40,42,44,37,39,39,38,4th,22,44
10,10,John Tyler,Independent,34,33,35,34,22,26,37,...,36,36,26,32,36,37,37,4th,22,41


In [162]:
# Calculate each row and column’s maximum and minimum value using the .apply method.

pres_app = pres.copy()

pres_app[['Min','Max']] = (pres_app
                            .select_dtypes(include='number')
                            .apply(['min','max'], axis=1))

pres_app

,Seq,President,Party,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,...,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank,Quartile,Min,Max
1,1,George Washington,Independent,7,7,1,10,1,6,2,...,1,2,2,1,2,1,1,1st,1,18
2,2,John Adams,Federalist,3,13,4,4,24,14,31,...,15,19,13,16,10,14,13,2nd,3,31
3,3,Thomas Jefferson,Democratic-Republican,2,2,14,1,8,5,14,...,4,6,9,7,5,5,5,1st,1,20
4,4,James Madison,Democratic-Republican,4,6,7,3,16,15,6,...,7,11,19,11,8,7,7,1st,3,19
5,5,James Monroe,Democratic-Republican,9,14,11,18,6,16,7,...,9,10,5,6,9,8,8,1st,5,18
6,6,John Quincy Adams,Democratic-Republican,1,9,6,5,29,19,24,...,18,21,15,14,18,18,18,2nd,1,29
7,7,Andrew Jackson,Democratic,37,15,29,28,4,4,38,...,25,17,23,20,19,19,19,2nd,4,38
8,8,Martin Van Buren,Democratic,23,22,27,25,34,28,20,...,26,29,27,24,28,25,25,3rd,16,34
9,9,William Henry Harrison,Whig,22,38,28,37,44,32,41,...,40,42,44,37,39,39,38,4th,22,44
10,10,John Tyler,Independent,34,33,35,34,22,26,37,...,36,36,26,32,36,37,37,4th,22,41
